## Jędrzej Nogacki 120673, Nowoczesne Technologie Przetwarzania Danych


### Temat: API do serwowania modelu ML, testowanie endpointów i zwracanie predykcji

### Zadanie 1: Stworzenie podstawowej aplikacji API

In [1]:
import subprocess
import time

# Uruchamiamy serwer FastAPI w tle
process = subprocess.Popen(["uvicorn", "app:app", "--host", "127.0.0.1", "--port", "8000", "--reload"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Poczekaj chwilę, aby upewnić się, że serwer się uruchomił
time.sleep(2)

print("Serwer FastAPI uruchomiony na http://127.0.0.1:8000")

Serwer FastAPI uruchomiony na http://127.0.0.1:8000


### Zadanie 2: Dodanie modelu ML i endpointu predykcji

In [3]:
# zaimportowanie danych oraz bibliotek użytych w modelu
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

# ładowanie danych do obiektu dataframe
data = load_wine(as_frame=True)
X = data.data
y = data.target
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# podział na zbiory oraz trenowanie modelu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# dodanie sztucznego szumu aby model się nie przetrenował
X_train += np.random.normal(0, 0.1, X_train.shape)
X_test += np.random.normal(0, 0.1, X_test.shape)

# standaryzacja danych
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

# uczenie modelu
model = LogisticRegression(max_iter=1000, C=0.01, random_state=42)
model.fit(X_scaled_train, y_train)
y_pred = model.predict(X_scaled_test)
acc = accuracy_score(y_test, y_pred)

print(f"Accuracy score of the model: {acc}")

Accuracy score of the model: 0.9722222222222222


In [11]:
# pakujemy model za pomocą biblioteki joblib aby wykorzystać go w serwerze
import joblib

model_name = 'wine_model.pkl'
scaler_name = 'scaler.pkl'

joblib.dump(model, model_name)
joblib.dump(scaler, scaler_name)

print(f"Model i scaler zapisane jako {model_name} i {scaler_name}")